Especialização em Análise e Engenharia de Dados - Cesar School

Disciplina: Estatística Descritiva dos Dados - Avaliação 2

Professor: Tsang Ing Ren

Monitor: José Ivson Soares da Silva

Alunos:

01. Cristiano Santana (Mat:3415)
02. Matheus Queiroz (Mat: 3376)
03. Nilton Coura (Mat: 3383)

Cristiano Santana (3415) | Nilton Coura (3376) | Matheus Queiroz (3383)


Fonte dos dados: arquivo divorse.csv disponível em https://raw.githubusercontent.com/joseivson/statistics/master/divorce.csv

LINK PARA O ARQUIVO (GITHUB)

https://github.com/MatheusBLQueiroz/estatistica

https://raw.githubusercontent.com/MatheusBLQueiroz/Esp/master/PRSA.csv 


AVALIAÇÃO 2: APLICAÇÃO DO TEOREMA DE BAYES PARA BASE DE DADOS "divorse.csv"

In [0]:
# Import das bibliotecas a serem utitlzadas no exercício e conexão com a base de dados 'divorse.csv'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

df = pd.read_csv('https://raw.githubusercontent.com/joseivson/statistics/master/divorce.csv', sep=';', decimal=',', thousands='.', encoding='ISO-8859-1')

In [0]:
# Apresentação inicial da base de dados 
df.head()

In [0]:
questions_list = ['Atr1','Atr2','Atr3','Atr4','Atr5','Atr6','Atr7','Atr8','Atr9','Atr10','Atr11','Atr12','Atr13','Atr14','Atr15','Atr16','Atr17','Atr18','Atr19','Atr20','Atr21','Atr22','Atr23','Atr24','Atr25','Atr26','Atr27','Atr28','Atr29','Atr30','Atr31','Atr32','Atr33','Atr34','Atr35','Atr36','Atr37','Atr38','Atr39','Atr40','Atr41','Atr42','Atr43','Atr44','Atr45','Atr46','Atr47','Atr48','Atr49','Atr50','Atr51','Atr52','Atr53','Atr54']

# A probabilidade a ser calculada terá como base 7 questões escolhidas de forma aleatória.
index_question = np.random.randint(low=0, high=53, size=7)
dataset = df.loc[:,['Atr'+str(index_question[0]), 'Atr'+str(index_question[1]), 'Atr'+str(index_question[2]), 'Atr'+str(index_question[3]), 'Atr'+str(index_question[4]), 'Atr'+str(index_question[5]), 'Atr'+str(index_question[6]), 'Class']]

print('index_question: ', index_question)

In [0]:
dataset.head()

In [0]:
dataset.shape

In [0]:
# Separate by class
def separate_by_class(dataset):
  classes = np.unique(dataset['Class'])

  separated = []
  for cl in classes:
    separated.append(dataset[dataset['Class'] == cl])

  return separated

separated = separate_by_class(dataset)

In [0]:
# Summarize data
# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset, eliminate_last=True):
  if eliminate_last:
    columns = dataset.columns[:-1]
  else:
    columns = dataset.columns
  summaries = [(np.mean(dataset[column]), np.std(dataset[column]), len(dataset[column])) for column in dataset.columns]
  del(summaries[-1])
  return summaries

summaries = summarize_dataset(dataset)

print(summaries)

In [0]:
# Summarize data by class
def summarize_by_class(dataset):
  separated = separate_by_class(dataset)
  summaries = dict()
  for s in separated:
    class_value = np.array(s['Class'])
    rows = s[s.columns]
    summaries[class_value[0]] = summarize_dataset(rows, eliminate_last=False)
  return summaries

summaries_by_class = summarize_by_class(dataset)
print(summaries_by_class)

In [0]:
# Gaussian probability
def calculate_probability(x, mean, stdev):
	exponent = np.exp(-((x-mean)**2 / (2 * stdev**2 )))
	return (1 / (np.sqrt(2 * np.pi) * stdev)) * exponent

In [0]:
 print(index_question)

In [0]:
# Probability distribution of "Divorse"
x = np.arange(-4.5,10,0.1)
for summ in summaries:
  y = calculate_probability(x, summ[0], summ[1])
  plt.plot(x,y)

plt.legend(['Atr'+str(index_question[0]), 'Atr'+str(index_question[1]), 'Atr'+str(index_question[2]), 'Atr'+str(index_question[3]), 'Atr'+str(index_question[4]), 'Atr'+str(index_question[5]), 'Atr'+str(index_question[6])])

In [0]:
# Probability distribution by class
n_classes = len(summaries_by_class)
fig,ax = plt.subplots(1,n_classes)
fig.set_size_inches(18, h=6)

classes_name = ['Casados', 'Não casados']
for c in range(n_classes):
  for summ in summaries_by_class[c]:
    ax[c].plot(x, calculate_probability(x, summ[0], summ[1]))
  ax[c].set_title(classes_name[c])

fig.legend(['Atr'+str(index_question[0]), 'Atr'+str(index_question[1]), 'Atr'+str(index_question[2]), 'Atr'+str(index_question[3]), 'Atr'+str(index_question[4]), 'Atr'+str(index_question[5]), 'Atr'+str(index_question[6])])


In [0]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [0]:
# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
  total_rows = sum([summaries[label][0][2] for label in summaries])
  probabilities = dict()
  for class_value, class_summaries in summaries.items():
    probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
    for i in range(len(class_summaries)):
      mean, stdev, count = class_summaries[i]
      probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
  return probabilities

In [0]:
# Predict the class for a given row
def predict(summaries, row):

	probabilities = calculate_class_probabilities(summaries, row)
	best_label, best_prob = None, -1
	for class_value, probability in probabilities.items():
		if best_label is None or probability > best_prob:
			best_prob = probability
			best_label = class_value
	return best_label

In [0]:
# Split the dataset
def split(dataset):
  sz = dataset.shape[0]
  sh = np.arange(sz)
  np.random.shuffle(sh)
  vec = np.zeros((sz),dtype=bool)
  vec[sh[:int(sz*0.75)]] = True
  train = dataset.loc[vec]
  test = dataset.loc[~vec]

  return train, test


In [0]:
train,test = split(dataset)

In [0]:
# Naive Bayes Algorithm
def naive_bayes(train, test):
  summarize = summarize_by_class(train)
  predictions = list()
  vec = np.zeros(test.shape[0], dtype=bool)
  for i in range(test.shape[0]):
    vec[i] = True
    row = np.array(test.loc[vec])[0]
    output = predict(summarize, row)
    predictions.append(output)
    vec[i] = False
  return(predictions)

In [0]:
predictions = naive_bayes(train,test)

print(predictions)
actual = np.array(test['Class'])
print(actual)

print('')
print('Resultado do cálculo da probalidade baseada na aplicação do Teorema de Bayes')
print('Accuracy >>>', accuracy_metric(actual, predictions))